# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import time

import numpy as np
import pandas as pd
import pickle

from tqdm.autonotebook import tqdm

# import oil.utils.Task268_download_metadata_from_CME as t268

/home/julia/.conda/envs/develop/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
import logging

_log = logging.getLogger()
_log.setLevel(logging.INFO)

In [3]:
# import oil.utils.helpers as helpers
# helpers.init_logger(logging.INFO)

In [4]:
from pylab import rcParams
rcParams['figure.figsize'] = (20, 5)

In [5]:
# import infra.helpers.telegram_notify.telegram_notify as tg

# tgn = tg.TelegramNotify()

In [14]:
def get_squash_cols(df):
    squash_cols = []
    for col_name in df.columns:
        if isinstance(col_name, int):
            if col_name > 0:
                squash_cols.append(col_name)
    return squash_cols


def squash_cols(df, cols):
    if cols == [1]:
        squashed_series = df[1]
    else:
        squashed_series = df[cols].fillna('').apply(" ".join, axis=1)
    return squashed_series


def add_links_as_rows(df):
    link_df = df[['link_1']].dropna()
    link_df.columns = [1]
    series = pd.concat([df, link_df.rename('{} Link'.format)])[[1]]
    return series


def get_row(df, idx):
    df = df.copy()
    df[1] = squash_cols(df, get_squash_cols(df))
    df = df[[1, 'link_1']]
    tr_df = add_links_as_rows(df).T
    tr_df.index = [idx]
    return tr_df


def rename_duplicate_cols(df):
    df = df.copy()
    dupe_mask = df.columns.duplicated(keep='first')
    duped_col_names = [
        f"{col_name}_{i}" for i, col_name in enumerate(df.columns[dupe_mask])
    ]
    new_index = np.array(df.columns)
    new_index[dupe_mask] = duped_col_names
    df.columns = new_index
    return df

# Read csv

In [15]:
pse = pd.read_csv('/data/prices/product_slate_export_20190904.csv')

In [16]:
pse.head()

,Clearing,Globex,Floor,Clearport,Product Name,Product Group,Sub Group,Category,Sub Category,Cleared As,Exchange,Volume,Open Interest,product_link
0,ED,GE,NaN,ED,Eurodollar Futures,Interest Rate,Stirs,NaN,NaN,Futures,CME,3104281,13213901,http://www.cmegroup.com/trading/interest-rates...
1,25,ZF,NaN,25,5-Year T-Note Futures,Interest Rate,US Treasury,NaN,NaN,Futures,CBOT,1166719,4162661,http://www.cmegroup.com/trading/interest-rates...
2,21,ZN,NaN,21,10-Year T-Note Futures,Interest Rate,US Treasury,NaN,NaN,Futures,CBOT,2181982,3656744,http://www.cmegroup.com/trading/interest-rates...
3,26,ZT,NaN,26,2-Year T-Note Futures,Interest Rate,US Treasury,NaN,NaN,Futures,CBOT,666626,3510507,http://www.cmegroup.com/trading/interest-rates...
4,ES,ES,NaN,ES,E-mini S&P 500 Futures,Equities,US Index,NaN,NaN,Futures,CME,2025324,2619124,http://www.cmegroup.com/trading/equity-index/u...


In [17]:
pse.shape

(1178, 14)

# Extract HTMLs

In [19]:
name_link = pse[['Product Name', 'product_link']].copy()

In [21]:
name_link = name_link.rename(columns={
    'Product Name': 'product_name'
}).set_index('product_name')

In [22]:
name_link.head()

,product_link
product_name,
Eurodollar Futures,http://www.cmegroup.com/trading/interest-rates...
5-Year T-Note Futures,http://www.cmegroup.com/trading/interest-rates...
10-Year T-Note Futures,http://www.cmegroup.com/trading/interest-rates...
2-Year T-Note Futures,http://www.cmegroup.com/trading/interest-rates...
E-mini S&P 500 Futures,http://www.cmegroup.com/trading/equity-index/u...


In [29]:
name_link.shape

(1178, 1)

In [32]:
np.unique(name_link.index).shape

(1178,)

# Load

In [97]:
product_names_htmls = {}
for name, link in tqdm(name_link.iterrows(), total=len(name_link)):
    time.sleep(1)
    product_names_htmls[name] = t268.load_html_to_df(link[0])

2019-09-04 05:58:27 PM load_html_to_df: INFO No tables were extracted from http://www.cmegroup.com/trading/energy/natural-gas/panhandle-natural-gas-fixed-price-swap_contract_specifications.html
2019-09-04 06:13:49 PM load_html_to_df: INFO No tables were extracted from http://www.cmegroup.com/trading/energy/crude-oil/wcs-cushing-argus-vs-wti-calendar%20-month_contract_specifications.html


In [98]:
# tgn.notify('Loaded htmls')

In [103]:
product_names_htmls['Eurodollar Futures']

,0,1,link_1
0,Contract Unit,"$2,500 x Contract IMM Index",None
1,Price Quotation,Contract IMM Index = 100 minus R R = three-mon...,None
2,Trading Hours,SUN - FRI: 5:00 p.m. - 4:00 p.m. CT,None
3,Minimum Price Fluctuation,Nearest expiring contract month: One quarter o...,None
4,Product Code,CME Globex: GECME ClearPort: EDClearing: ED,None
5,Listed Contracts,"Nearest 40 months (i.e., 10 years) in the Marc...",None
6,Settlement Method,Financially Settled,None
7,Termination Of Trading,Second London bank business day before 3rd Wed...,None
8,Settlement Procedures,Eurodollar Future Settlement Procedures,http://www.cmegroup.com/confluence/display/EPI...
9,Position Limits,CME Position Limits,http://www.cmegroup.com/rulebook/files/positio...


# Dump

In [101]:
with open('/data/prices/Task268_contract_specs_20190904.pkl', 'wb') as fout:
    pickle.dump(product_names_htmls, fout)

# Add contract specs as rows

## Read the data

In [6]:
pse = pd.read_csv('/data/prices/product_slate_export_20190904.csv')

In [7]:
with open('/data/prices/Task268_contract_specs_20190904.pkl', 'rb') as fin:
    product_names_htmls = pickle.load(fin)

In [8]:
pse.head()

,Clearing,Globex,Floor,Clearport,Product Name,Product Group,Sub Group,Category,Sub Category,Cleared As,Exchange,Volume,Open Interest,product_link
0,ED,GE,NaN,ED,Eurodollar Futures,Interest Rate,Stirs,NaN,NaN,Futures,CME,3104281,13213901,http://www.cmegroup.com/trading/interest-rates...
1,25,ZF,NaN,25,5-Year T-Note Futures,Interest Rate,US Treasury,NaN,NaN,Futures,CBOT,1166719,4162661,http://www.cmegroup.com/trading/interest-rates...
2,21,ZN,NaN,21,10-Year T-Note Futures,Interest Rate,US Treasury,NaN,NaN,Futures,CBOT,2181982,3656744,http://www.cmegroup.com/trading/interest-rates...
3,26,ZT,NaN,26,2-Year T-Note Futures,Interest Rate,US Treasury,NaN,NaN,Futures,CBOT,666626,3510507,http://www.cmegroup.com/trading/interest-rates...
4,ES,ES,NaN,ES,E-mini S&P 500 Futures,Equities,US Index,NaN,NaN,Futures,CME,2025324,2619124,http://www.cmegroup.com/trading/equity-index/u...


In [9]:
product_names_htmls['Eurodollar Futures']

,0,1,link_1
0,Contract Unit,"$2,500 x Contract IMM Index",None
1,Price Quotation,Contract IMM Index = 100 minus R R = three-mon...,None
2,Trading Hours,SUN - FRI: 5:00 p.m. - 4:00 p.m. CT,None
3,Minimum Price Fluctuation,Nearest expiring contract month: One quarter o...,None
4,Product Code,CME Globex: GECME ClearPort: EDClearing: ED,None
5,Listed Contracts,"Nearest 40 months (i.e., 10 years) in the Marc...",None
6,Settlement Method,Financially Settled,None
7,Termination Of Trading,Second London bank business day before 3rd Wed...,None
8,Settlement Procedures,Eurodollar Future Settlement Procedures,http://www.cmegroup.com/confluence/display/EPI...
9,Position Limits,CME Position Limits,http://www.cmegroup.com/rulebook/files/positio...


## Investigate the data

In [10]:
pd.Series([
    contract_spec.shape[0] if contract_spec is not None else None
    for contract_spec in product_names_htmls.values()
]).value_counts()

13.0    659
15.0    178
16.0    128
14.0    104
12.0     60
17.0     38
18.0      8
11.0      1
dtype: int64

In [28]:
pd.Series([
    contract_spec.shape[1] if contract_spec is not None else None
    for contract_spec in product_names_htmls.values()
]).value_counts()

3.0    716
4.0    459
5.0      1
dtype: int64

In [26]:
for contract_spec in product_names_htmls.values():
    if contract_spec is not None:
        if contract_spec.shape[1] > 3:
            display(contract_spec)

,0,1,2,link_1
0,Contract Unit,"10,000 million British thermal units (MMBtu)",NaN,None
1,Price Quotation,U.S. dollars and cents per MMBtu,NaN,None
2,Trading Hours,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,NaN,None
3,Minimum Price Fluctuation,CME ClearPort:,0.001 per MMBtu = $10.00,None
4,Minimum Price Fluctuation,CME Globex:,0.001 per MMBtu = $10.00 Inter-commodity sprea...,None
5,Product Code,CME Globex: NGCME ClearPort: NGClearing: NGTAS...,NaN,None
6,Listed Contracts,Monthly contracts listed for the current year ...,NaN,None
7,Settlement Method,Deliverable,NaN,None
8,Termination Of Trading,Trading terminates on the 3rd last business da...,NaN,None
9,Trade At Marker Or Trade At Settlement Rules,Trading at settlement is available for spot (e...,NaN,None


,0,1,2,link_1
0,Contract Unit,"42,000 gallons",NaN,None
1,Price Quotation,U.S. dollars and cents per gallon,NaN,None
2,Trading Hours,CME Globex:,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,None
3,Trading Hours,Minimum Price Fluctuation,$0.0001 per gallon,None
4,Product Code,CME Globex: HOCME ClearPort: HOClearing: HOTAS...,NaN,None
5,Listed Contracts,Current Year + 3 Years + 1 Month,NaN,None
6,Settlement Method,Deliverable,NaN,None
7,Termination Of Trading,Trading in a current month shall cease on the ...,NaN,None
8,Trade At Marker Or Trade At Settlement Rules,Trading at settlement is available for spot (e...,NaN,None
9,Settlement Procedures,Heating Oil Futures Settlement Procedures,NaN,http://www.cmegroup.com/confluence/display/EPI...


,0,1,2,link_1
0,Contract Unit,"40,000 pounds (~18 metric tons)",NaN,None
1,Price Quotation,U.S. cents per pound,NaN,None
2,Trading Hours,CME ClearPort:,Monday - Friday: 8:30 a.m. - 1:05 p.m. CT,None
3,Trading Hours,Minimum Price Fluctuation,0.00025 per pound = $10.00,None
4,Product Code,CME Globex: LECME ClearPort: 48Clearing: 48TAS...,NaN,None
5,Listed Contracts,"9 monthly contracts of Feb, Apr, Jun, Aug, Oct...",NaN,None
6,Settlement Method,Deliverable,NaN,None
7,Termination Of Trading,Trading terminates at 12:00 Noon CT on the las...,NaN,None
8,Trade At Marker Or Trade At Settlement Rules,Trading at settlement is available for the fir...,NaN,None
9,Settlement Procedures,Live Cattle Settlement Procedures,NaN,http://www.cmegroup.com/confluence/display/EPI...


,0,1,2,link_1
0,Contract Unit,"10,000 MMBtu",NaN,None
1,Price Quotation,U.S. dollars and cents per MMBtu,NaN,None
2,Trading Hours,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,NaN,None
3,Minimum Price Fluctuation,CME ClearPort:,0.001 per MMBtu = $10.00,None
4,Minimum Price Fluctuation,CME Globex:,$0.001 per MMBtu = $10.00 Inter-commodity spre...,None
5,Product Code,CME Globex: HHCME ClearPort: HHClearing: HHTAS...,NaN,None
6,Listed Contracts,Monthly contracts listed for the current year ...,NaN,None
7,Settlement Method,Financially Settled,NaN,None
8,Floating Price,The Floating Price for each contract month wil...,NaN,None
9,Termination Of Trading,Trading terminates on the third last business ...,NaN,None


,0,1,2,link_1
0,Contract Unit,"25,000 pounds",NaN,None
1,Price Quotation,U.S. dollars and cents per pound,NaN,None
2,Trading Hours,CME Globex:,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,None
3,Trading Hours,Minimum Price Fluctuation,0.0005 per pound = $12.50,None
4,Product Code,CME Globex: HGCME ClearPort: HGClearing: HGTAS...,NaN,None
5,Listed Contracts,Monthly contracts listed for 24 consecutive mo...,NaN,None
6,Settlement Method,Deliverable,NaN,None
7,Termination Of Trading,Trading terminates at 12:00 Noon CT on the thi...,NaN,None
8,Trade At Marker Or Trade At Settlement Rules,Trading at Settlement is available in the spot...,NaN,None
9,Settlement Procedures,Copper Futures Settlement Procedures,NaN,http://www.cmegroup.com/confluence/display/EPI...


...

,0,1,2,link_1
0,Contract Unit,25 metric tons,NaN,None
1,Price Quotation,U.S. dollars and cents per metric ton,NaN,None
2,Trading Hours,CME Globex:,Sunday - Friday 7:00 p.m. - 7:45 a.m. (8:00 p....,None
3,Trading Hours,Minimum Price Fluctuation,0.50 per metric tons = $12.50,None
4,Product Code,CME Globex: BSFCME ClearPort: BSFClearing: BSF,NaN,None
5,Listed Contracts,Monthly contracts listed for 12 consecutive mo...,NaN,None
6,Settlement Method,Financially Settled,NaN,None
7,Floating Price,The Floating Price for each contract month sha...,NaN,None
8,Termination Of Trading,Trading terminates on the 15th calendar day of...,NaN,None
9,Position Limits,CBOT Position Limits,NaN,http://http://www.cmegroup.com/rulebook/files/...


,0,1,2,link_1
0,Contract Unit,5 MW (80 MWh) 5 MW x 16 peak hours = 80 MWh Cl...,NaN,None
1,Price Quotation,U.S. dollars and cents per MWh,NaN,None
2,Trading Hours,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,NaN,None
3,Minimum Price Fluctuation,0.05 x 5 MW x 16 peak hours x # peak days/month,NaN,None
4,Product Code,CME Globex: AOHCME ClearPort: OMClearing: OM,NaN,None
5,Listed Contracts,Monthly contracts listed for the current year ...,NaN,None
6,Settlement Method,Financially Settled,NaN,None
7,Floating Price,A Daily Floating Price will be determined for ...,NaN,None
8,Termination Of Trading,CME Globex:,Trading on CME Globex terminates at 23:59 EPT ...,None
9,Termination Of Trading,CME ClearPort:,Trading terminates on the last business day of...,None


,0,1,2,link_1
0,Contract Unit,"30,000,000 Hungarian forint",NaN,None
1,Trading Hours,CME Globex:,Sundays: 5:00 p.m. – 4:00 p.m. Central Time (C...,None
2,Trading Hours,Minimum Price Fluctuation,$.0000002 euro per Hungarian forint increments...,None
3,Product Code,CME Globex: EHFCME ClearPort: RClearing: R,NaN,None
4,Listed Contracts,"Six months in the March quarterly cycle (Mar, ...",NaN,None
5,Settlement Method,Deliverable,NaN,None
6,Termination Of Trading,9:16 a.m. Central Time (CT) on the second busi...,NaN,None
7,Settlement Procedures,HUF/EUR Settlement Procedures,NaN,http://www.cmegroup.com/confluence/display/EPI...
8,Position Limits,CME Position Limits,NaN,http://www.cmegroup.com/rulebook/files/positio...
9,Exchange Rulebook,CME 316,NaN,http://www.cmegroup.com/rulebook/CME/III/300/3...


,0,1,2,link_1
0,Contract Unit,5 MW x 16 peak hours = 80 MWh,NaN,None
1,Price Quotation,U.S. dollars and cents per MWh,NaN,None
2,Trading Hours,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,NaN,None
3,Minimum Price Fluctuation,0.01 x 5 MW x 16 peak hours = $0.80,NaN,None
4,Product Code,CME Globex: AK1CME ClearPort: K1Clearing: K1,NaN,None
5,Listed Contracts,Daily contracts listed for the current month a...,NaN,None
6,Settlement Method,Financially Settled,NaN,None
7,Floating Price,The Daily Floating Price will be determined fo...,NaN,None
8,Termination Of Trading,CME Globex:,Trading on CME Globex terminates at 23:59 CPT ...,None
9,Termination Of Trading,CME ClearPort:,Trading terminates on the contract day. If the...,None


,0,1,2,link_1
0,Contract Unit,"42,000 gallons",NaN,None
1,Price Quotation,U.S. dollars and cents per gallon,NaN,None
2,Trading Hours,CME Globex:,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,None
3,Trading Hours,Minimum Price Fluctuation,$0.0001 per gallon,None
4,Product Code,CME Globex: CBACME ClearPort: CBAClearing: CBA,NaN,None
5,Listed Contracts,36 consecutive calendar months,NaN,None
6,Settlement Method,Financially Settled,NaN,None
7,Floating Price,The Floating Price for each contract month is ...,NaN,None
8,Termination Of Trading,Trading shall cease on the last business day o...,NaN,None
9,Position Limits,NYMEX Position Limits,NaN,http://www.cmegroup.com/rulebook/files/positio...


In [29]:
for contract_spec in product_names_htmls.values():
    if contract_spec is not None:
        if contract_spec.shape[1] == 5:
            display(contract_spec)

,0,1,2,3,link_1
0,Contract Unit,25 Metric Tons,NaN,NaN,None
1,Price Quotation,U.S. Dollars and Cents per Metric Ton,NaN,NaN,None
2,Trading Hours,CME Globex:,"Monday – Friday, 8:30 a.m. – 1:20 p.m.CT",NaN,None
3,Trading Hours,Minimum Price Fluctuation,CME ClearPort:,$0.25 per Metric Ton,None
4,CME Globex:,Minimum Price Fluctuation,$0.250 per Metric Ton,NaN,None
5,Product Code,CME Globex: POGCME ClearPort: POGClearing: POG,NaN,NaN,None
6,Listed Contracts,Monthly contracts listed for 12 consecutive mo...,NaN,NaN,None
7,Settlement Method,Financially Settled,NaN,NaN,None
8,Termination Of Trading,Trading terminates on the last Bursa Malaysia ...,NaN,NaN,None
9,Position Limits,CME Position Limits,NaN,NaN,http://www.cmegroup.com/rulebook/files/positio...


In [32]:
sum([
    'link_1' in contract_spec.columns if contract_spec is not None else True
    for contract_spec in product_names_htmls.values()
])

1178

In [33]:
len(product_names_htmls)

1178

## Add contract specs

In [10]:
product_names_htmls = {
    product_name: html.set_index(0) if html is not None else None
    for product_name, html in product_names_htmls.items()
}

In [11]:
product_names_htmls['Eurodollar Futures']

,1,link_1
0,,
Contract Unit,"$2,500 x Contract IMM Index",None
Price Quotation,Contract IMM Index = 100 minus R R = three-mon...,None
Trading Hours,SUN - FRI: 5:00 p.m. - 4:00 p.m. CT,None
Minimum Price Fluctuation,Nearest expiring contract month: One quarter o...,None
Product Code,CME Globex: GECME ClearPort: EDClearing: ED,None
Listed Contracts,"Nearest 40 months (i.e., 10 years) in the Marc...",None
Settlement Method,Financially Settled,None
Termination Of Trading,Second London bank business day before 3rd Wed...,None
Settlement Procedures,Eurodollar Future Settlement Procedures,http://www.cmegroup.com/confluence/display/EPI...


In [15]:
html_rows = [
    get_row(html, product_name) if html is not None else pd.DataFrame(
        index=[product_name])
    for product_name, html in product_names_htmls.items()
]

In [16]:
html_rows[0]

,Contract Unit,Price Quotation,Trading Hours,Minimum Price Fluctuation,Product Code,Listed Contracts,Settlement Method,Termination Of Trading,Settlement Procedures,Position Limits,...,Price Limit Or Circuit,All Or None Minimum,Vendor Codes,Settlement Procedures Link,Position Limits Link,Exchange Rulebook Link,Block Minimum Link,Price Limit Or Circuit Link,All Or None Minimum Link,Vendor Codes Link
Eurodollar Futures,"$2,500 x Contract IMM Index",Contract IMM Index = 100 minus R R = three-mon...,SUN - FRI: 5:00 p.m. - 4:00 p.m. CT,Nearest expiring contract month: One quarter o...,CME Globex: GECME ClearPort: EDClearing: ED,"Nearest 40 months (i.e., 10 years) in the Marc...",Financially Settled,Second London bank business day before 3rd Wed...,Eurodollar Future Settlement Procedures,CME Position Limits,...,Price Limits,All or None Minimums,Quote Vendor Symbols Listing,http://www.cmegroup.com/confluence/display/EPI...,http://www.cmegroup.com/rulebook/files/positio...,http://www.cmegroup.com/rulebook/CME/V/450/452...,http://www.cmegroup.com/clearing/trading-pract...,http://www.cmegroup.com/rulebook/files/special...,http://www.cmegroup.com/clearing/trading-pract...,http://www.cmegroup.com/tools-information/vend...


In [17]:
html_rows = [
    get_row(html, product_name) if html is not None else pd.DataFrame(
        index=[product_name])
    for product_name, html in product_names_htmls.items()
]

In [18]:
html_rows[8]

,Contract Unit,Price Quotation,Trading Hours,Minimum Price Fluctuation,Minimum Price Fluctuation,Product Code,Listed Contracts,Settlement Method,Termination Of Trading,Trade At Marker Or Trade At Settlement Rules,...,Block Minimum,Price Limit Or Circuit,Vendor Codes,Grade And Quality,Settlement Procedures Link,Position Limits Link,Exchange Rulebook Link,Block Minimum Link,Price Limit Or Circuit Link,Vendor Codes Link
Henry Hub Natural Gas Futures,"10,000 million British thermal units (MMBtu)",U.S. dollars and cents per MMBtu,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,CME ClearPort: 0.001 per MMBtu = $10.00,CME Globex: 0.001 per MMBtu = $10.00 Inter-com...,CME Globex: NGCME ClearPort: NGClearing: NGTAS...,Monthly contracts listed for the current year ...,Deliverable,Trading terminates on the 3rd last business da...,Trading at settlement is available for spot (e...,...,Block Minimum Thresholds,Price Limits,Quote Vendor Symbols Listing,Natural Gas meeting the specifications set for...,http://www.cmegroup.com/confluence/display/EPI...,http://www.cmegroup.com/rulebook/files/positio...,http://www.cmegroup.com/rulebook/NYMEX/2/220.pdf,http://www.cmegroup.com/clearing/trading-pract...,https://www.cmegroup.com/trading/price-limits....,http://www.cmegroup.com/tools-information/vend...


In [19]:
rename_duplicate_cols(html_rows[8])

,Contract Unit,Price Quotation,Trading Hours,Minimum Price Fluctuation,Minimum Price Fluctuation_0,Product Code,Listed Contracts,Settlement Method,Termination Of Trading,Trade At Marker Or Trade At Settlement Rules,...,Block Minimum,Price Limit Or Circuit,Vendor Codes,Grade And Quality,Settlement Procedures Link,Position Limits Link,Exchange Rulebook Link,Block Minimum Link,Price Limit Or Circuit Link,Vendor Codes Link
Henry Hub Natural Gas Futures,"10,000 million British thermal units (MMBtu)",U.S. dollars and cents per MMBtu,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,CME ClearPort: 0.001 per MMBtu = $10.00,CME Globex: 0.001 per MMBtu = $10.00 Inter-com...,CME Globex: NGCME ClearPort: NGClearing: NGTAS...,Monthly contracts listed for the current year ...,Deliverable,Trading terminates on the 3rd last business da...,Trading at settlement is available for spot (e...,...,Block Minimum Thresholds,Price Limits,Quote Vendor Symbols Listing,Natural Gas meeting the specifications set for...,http://www.cmegroup.com/confluence/display/EPI...,http://www.cmegroup.com/rulebook/files/positio...,http://www.cmegroup.com/rulebook/NYMEX/2/220.pdf,http://www.cmegroup.com/clearing/trading-pract...,https://www.cmegroup.com/trading/price-limits....,http://www.cmegroup.com/tools-information/vend...


In [20]:
html_rows = [rename_duplicate_cols(html) for html in html_rows]

In [21]:
html_rows_df = pd.concat(html_rows, sort=False)

In [22]:
html_rows_df.head()

,Contract Unit,Price Quotation,Trading Hours,Minimum Price Fluctuation,Product Code,Listed Contracts,Settlement Method,Termination Of Trading,Settlement Procedures,Position Limits,...,Delivery Procedure Link,Trade At Marker Or Trade At Settlement Rules Link,Minimum Price Fluctuation Link,Listed Contracts Link,Termination Of Trading_0,Termination Of Trading Link,Reportable Positions,Termination Of Trading_1,CME Globex:,Reportable Positions Link
Eurodollar Futures,"$2,500 x Contract IMM Index",Contract IMM Index = 100 minus R R = three-mon...,SUN - FRI: 5:00 p.m. - 4:00 p.m. CT,Nearest expiring contract month: One quarter o...,CME Globex: GECME ClearPort: EDClearing: ED,"Nearest 40 months (i.e., 10 years) in the Marc...",Financially Settled,Second London bank business day before 3rd Wed...,Eurodollar Future Settlement Procedures,CME Position Limits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5-Year T-Note Futures,"Face value at maturity of $100,000",Points and fractions of points with par on the...,SUN - FRI: 5:00 p.m. - 4:00 p.m.,One-quarter of one thirty-second (1/32) of one...,CME Globex: ZFCME ClearPort: 25Clearing: 25,The first three consecutive contracts in the M...,Deliverable,Last business day of the calendar month. Tradi...,Treasury Settlement Procedures,CBOT Position Limits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10-Year T-Note Futures,"Face value at maturity of $100,000",Points and fractions of points with par on the...,Sunday - Friday 6:00 p.m. - 5:00 p.m. (5:00 p....,Outrights: One-half of one thirty-second (1/32...,CME Globex: ZNCME ClearPort: 21Clearing: 21,"Quarterly contracts (Mar, Jun, Sep, Dec) liste...",Deliverable,Trading terminates on the seventh business day...,Treasury Settlement Procedures,CBOT Position Limits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2-Year T-Note Futures,"Face value at maturity of $200,000",Points and fractions of points with par on the...,SUN - FRI: 5:00 p.m. - 4:00 p.m.,One-eighth of one thirty-second (1/32) of one ...,CME Globex: ZTCME ClearPort: 26Clearing: 26,The first three consecutive contracts in the M...,Deliverable,Last business day of the calendar month. Tradi...,Treasury Settlement Procedures,CBOT Position Limits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E-mini S&P 500 Futures,$50 x S&P 500 Index,NaN,CME Globex: Sunday - Friday 6:00 p.m. - 5:00 p...,Outright: 0.25 index points = $12.50 Calendar ...,CME Globex: ESCME ClearPort: ESClearing: ESBTI...,"Quarterly contracts (Mar, Jun, Sep, Dec) liste...",Financially Settled,Trading terminates at 9:30 a.m. ET on the 3rd ...,Settlement Procedures,CME Position Limits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
pse_with_specs = pse.merge(html_rows_df,
                           left_on='Product Name',
                           right_index=True)

In [24]:
pse_with_specs.to_csv(
    '/data/prices/product_slate_export_with_contract_specs_20190905.csv', index=0)